In [1]:
#preamble
import pandas as pd
import numpy as np
import matplotlib as mt
import glob as glob

#downloading files
filenames = glob.glob('Combined_*[a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p]*160c.dta')
list_of_all_dfs  = [pd.read_stata(f) for f in filenames]
list_of_all_dfs = [list_of_all_dfs[i].set_index('xwaveid') for i in range(16)]

#pulling out variables of interest 
prev = [list_of_all_dfs[i].filter(regex='jbhrcpr$') for i in range(16)]
sex = [list_of_all_dfs[i].filter(regex='hgsex$') for i in range(16)]
job = [list_of_all_dfs[i].filter(regex='esbrd$') for i in range(16)]
hrs_act = [list_of_all_dfs[i].filter(regex='jbhruc$') for i in range(16)]
hrs_id = [list_of_all_dfs[i].filter(regex='jbtprhr$') for i in range(16)]

# making into dataframes 
prev = [pd.DataFrame(prev[i]) for i in range(16)]
sex = [pd.DataFrame(sex[i]) for i in range(16)]
job = [pd.DataFrame(job[i]) for i in range(16)]
hrs_act = [pd.DataFrame(hrs_act[i]) for i in range(16)]
hrs_id = [pd.DataFrame(hrs_id[i]) for i in range(16)]

In [2]:
# merging into one dataframe for simplicity 
result1 = [pd.merge(sex[i], job[i], left_index=True, right_index=True) for i in range(16)]
result1 = [pd.merge(result1[i], prev[i], left_index=True, right_index=True) for i in range(16)]
result1 = [pd.merge(result1[i], hrs_id[i], left_index=True, right_index=True) for i in range(16)]
result1 = [pd.merge(result1[i], hrs_act[i], left_index=True, right_index=True) for i in range(16)]

In [3]:
# narrowing results 
# this is now all employed respondants that want more/less hours 
pt = [result1[i][result1[i].iloc[:, 1] == '[1] Employed'] for i in range(16)]
pt = [pt[i][(pt[i].iloc[:, 2] == '[3] More hours') | (pt[i].iloc[:, 2] == '[1] Fewer hours')] for i in range(16)]
# narrowing to just men 
men = [pt[i][pt[i].iloc[:, 0] == '[1] Male'] for i in range(16)]
#looking at those who prefer to work fewer
men_few = [men[i][men[i].iloc[:, 2] == '[1] Fewer hours'] for i in range(16)]

In [4]:
#pulling out columns of interest 
des1 = [men_few[i].filter(regex='jbtprhr$') for i in range(16)]
act1 = [men_few[i].filter(regex='jbhruc$') for i in range(16)]

#turning into df
merged = [pd.merge(des1[i], act1[i], left_index=True, right_index=True) for i in range(16)]
merged1 = [pd.DataFrame(merged[i]) for i in range(16)]


# pulling out important variables, making numeric
des11 = [merged1[i].iloc[:,0] for i in range(16)]
act11 = [merged1[i].iloc[:,1] for i in range(16)]
numb_des = [pd.to_numeric(des11[i], errors = 'coerce') for i in range(16)]
numb_act = [pd.to_numeric(act11[i], errors = 'coerce') for i in range(16)]

#merging numeric columns, then dropping na responses 
check = [pd.merge(numb_des[i], numb_act[i], left_index=True, right_index=True) for i in range(16)]
check_dropped = [check[i].dropna() for i in range(16)]
few_n = [len(check_dropped[i]) for i in range(16)]

# looping over to create difference column
for i in range(16):
    check_dropped[i]['dif'] = check_dropped[i].iloc[:,0] - check_dropped[i].iloc[:,1]

new_mean= [check_dropped[i].iloc[:,2].mean() for i in range(16)]
diff_few = pd.DataFrame([new_mean[0], new_mean[1], new_mean[2], new_mean[3], new_mean[4], new_mean[5], new_mean[6], new_mean[7], new_mean[8], new_mean[9], new_mean[10], new_mean[11], new_mean[12], new_mean[13], new_mean[14], new_mean[15]])

# getting mean of difference column
diff_mean = diff_few.mean()
diff_end = pd.concat([diff_few, diff_mean], axis=0)
diff_end = round(diff_end, 1)


#creating totals column
few_totals = pd.DataFrame([few_n[0], few_n[1], few_n[2], few_n[3], few_n[4], few_n[5], few_n[6], few_n[7], few_n[8], few_n[9], few_n[10], few_n[11], few_n[12], few_n[13], few_n[14], few_n[15]])
few_totals11 = pd.DataFrame(few_totals.sum())
few_totals_final = pd.concat([few_totals, few_totals11], axis=0)
few_table = pd.concat([few_totals_final, diff_end], axis=1)



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# will now repeat this for those who prefer to work more 
# looking at those who prefer to work more 
men_mo = [men[i][men[i].iloc[:, 2] == '[3] More hours'] for i in range(16)]

# putting into columns of interest 
des_mo = [men_mo[i].filter(regex='jbtprhr$') for i in range(16)]
act_mo = [men_mo[i].filter(regex='jbhruc$') for i in range(16)]

#merging into one frame, turning into a dataframe
mo_merged = [pd.merge(des_mo[i], act_mo[i], left_index=True, right_index=True) for i in range(16)]
mo_merged1 = [pd.DataFrame(mo_merged[i]) for i in range(16)]

#pulling out important variables, making numeric

numb_des = [mo_merged1[i].iloc[:,0] for i in range(16)]
numb_act = [mo_merged1[i].iloc[:,1] for i in range(16)]
numb_des = [pd.to_numeric(numb_des[i], errors = 'coerce') for i in range(16)]
numb_act = [pd.to_numeric(numb_act[i], errors = 'coerce') for i in range(16)]

#merging numeric columns, then dropping na responses 
checkmo = [pd.merge(numb_des[i], numb_act[i], left_index=True, right_index=True) for i in range(16)]
checkmo_dropped = [checkmo[i].dropna() for i in range(16)]
mo_n = [len(checkmo_dropped[i]) for i in range(16)]


# looping over to create difference column
for i in range(16):
    checkmo_dropped[i]['dif'] = checkmo_dropped[i].iloc[:,0] - checkmo_dropped[i].iloc[:,1]

new_mean= [checkmo_dropped[i].iloc[:,2].mean() for i in range(16)]
diff_mo = pd.DataFrame([new_mean[0], new_mean[1], new_mean[2], new_mean[3], new_mean[4], new_mean[5], new_mean[6], new_mean[7], new_mean[8], new_mean[9], new_mean[10], new_mean[11], new_mean[12], new_mean[13], new_mean[14], new_mean[15]])

# getting mean of difference column
mo_mean = diff_mo.mean()
mo_end = pd.concat([diff_mo, mo_mean], axis=0)
mo_end = round(mo_end, 1)

#creating totals column
mo_totals = pd.DataFrame([mo_n[0], mo_n[1], mo_n[2], mo_n[3], mo_n[4], mo_n[5], mo_n[6], mo_n[7], mo_n[8], mo_n[9], mo_n[10], mo_n[11], mo_n[12], mo_n[13], mo_n[14], mo_n[15]])
mo_totals11 = pd.DataFrame(mo_totals.sum())
mo_totals_final = pd.concat([mo_totals, mo_totals11], axis=0)
mo_table = pd.concat([mo_totals_final, mo_end], axis=1)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
# Creating a table for the average of any mismatch 

#pulling out variables of interest 
des_all = [men[i].filter(regex='jbtprhr$') for i in range(16)]
act_all = [men[i].filter(regex='jbhruc$') for i in range(16)]
all_merged = [pd.merge(des_all[i], act_all[i], left_index=True, right_index=True) for i in range(16)]

#dropping non-responses 
drop_all = [all_merged[i].dropna() for i in range(16)]
des_ = [drop_all[i].iloc[:,0] for i in range(16)]
des_all1 = [pd.to_numeric(des_[i], errors = 'coerce') for i in range(16)]

# getting the mean
des_alltot = [len(des_all1[i]) for i in range(16)]
act_ = [drop_all[i].iloc[:,1] for i in range(16)]

# turning into numbers, getting mean and total 
act_all1 = [pd.to_numeric(act_[i], errors = 'coerce') for i in range(16)]
act_allmean = [act_all1[i].mean() for i in range(16)]


#merging the two numerical columns, then dropping na values for both columns
test = [pd.merge(des_all1[i], act_all1[i], left_index=True, right_index=True) for i in range(16)]
test_total = [len(test[i]) for i in range(16)]

for i in range(16):
    test[i]['dif'] = test[i].iloc[:,0] - test[i].iloc[:,1]

new_mean= [test[i].iloc[:,2].mean() for i in range(16)]
diff_all = pd.DataFrame([new_mean[0], new_mean[1], new_mean[2], new_mean[3], new_mean[4], new_mean[5], new_mean[6], new_mean[7], new_mean[8], new_mean[9], new_mean[10], new_mean[11], new_mean[12], new_mean[13], new_mean[14], new_mean[15]])

# getting mean of difference column
all_mean = diff_all.mean()
all_end = pd.concat([diff_all, all_mean], axis=0)
all_end = round(all_end, 1)

all_totals = pd.DataFrame([test_total[0], test_total[1], test_total[2], test_total[3], test_total[4], test_total[5], test_total[6], test_total[7], test_total[8], test_total[9], test_total[10], test_total[11], test_total[12], test_total[13], test_total[14], test_total[15]])
all_totals11 = pd.DataFrame(all_totals.sum())
all_totals_final = pd.concat([all_totals, all_totals11], axis=0)
all_table = pd.concat([all_totals_final, all_end], axis=1)


In [36]:
table4 = pd.concat([few_table, mo_table, all_table], axis=1)

In [38]:
table4.columns = ['Number of workers', 'Average hours', 'Number of workers', 'Average hours', 'Number of workers', 'Average hours']

In [39]:
table4.index = [ '2001 (1)', '2002 (2)', '2003 (3)', '2004 (4)', '2005 (5)', '2006 (6)', '2007 (7)', '2008 (8)', '2009 (9)', '2010 (10)', '2011 (11)', '2012 (12)', '2013 (13)', '2014 (14)', '2015 (15)', '2016 (16)', 'All']


In [40]:
table4

,Number of workers,Average hours,Number of workers,Average hours,Number of workers,Average hours
2001 (1),1372,-14.7,691,11.7,2065,-5.9
2002 (2),1352,-14.7,652,11.8,2005,-6.1
2003 (3),1326,-14.5,579,12.4,1913,-6.3
2004 (4),1282,-14.6,551,12.0,1835,-6.6
2005 (5),1274,-14.4,564,11.9,1841,-6.3
2006 (6),1285,-14.6,569,12.4,1858,-6.3
2007 (7),1233,-14.4,514,11.8,1750,-6.7
2008 (8),1204,-14.5,530,11.4,1736,-6.6
2009 (9),1226,-14.6,629,12.4,1863,-5.4
2010 (10),1153,-14.0,621,11.6,1779,-5.0
